# Defining Variables in TensorFlow
* Variable is essentially a tensor with a specific shape defining how many dimensions the variable will have and the size of each dimension. 
* Unlike regular tensor, variables are mutable - ideal property to implement parameters of learning models where the weights change slightly after each step of learning.
* If you define `tf.Variable(0, dtype=tf.int32)`, you can change its value using a TensorFlow operation such as `tf.assign(x, x+1)`. 
* Cannot change value of a defined tensor such as `tf.constant(0, dtype=tf.int32)`. It should stay 0 unitl the end of the program execution. 
* A few things are of high importance when creating variables:
1. Variable shape
2. Data type
3. Initial value
4. Name (optional)

### Variable Shape
* 1D vector of the `[x, y, z, ...]` format. 
* Each value indicates how large corresponding dimension or axis is. 
* e.g. 2D tensor with 50 rows and 10 columns would have shape `[50, 10]`.
* Dimensionality of the variable (length of the `shape` vector) is recognized as the rank of the tensor in TensorFlow.

### Data Type
* Plays an important role in determining size of variable. 
* Commonly used include `tf.bool, tf.uint8, tf.float32, tf.int32`. 
* Each data type has a number of bits required to represent a single value with that type. e.g. `tf.uint8` requires 8 bits, `tf.float32` requires 32 bits.
* Common practice to use same data types for computations to avoid mismatches. 
* If you have two different data types for two tensors that you need to transform, you have to explicitly convert one tensor to the other tensor's type using the `tf.cast(...)` operation.
* e.g. you have an `x` variable with type `tf.int32` which needs to be converted to `tf.float32`, we would use `tf.cast(x, dtype=float32)`. 

### Initial Value
* Variable requires initial value to be initialized with. 
* TensorFlow provides several, including constant & normal distribution initializers.
Few popular TensorFlow initializers:
1. `tf.zeros`
2. `tf.constant_initializer`
3. `tf.random_uniform`
4. `tf.truncated_normal`

### Name
* Used as an ID to identify variable in graph. 
* If you ever visualize graph, name will appear by the argument passed to the `name` keyword. 
* If no name is specified, TensorFlow will use default naming scheme. 

# Defining TensorFlow outputs
* Usually tensors and a result of a transformation to either an input or a variable or both. 
* We can pass outputs to other operations. 
* We can also use standard Python arithmetic.
```
x = tf.matmul (w, A)
y - x + B
z = tf.add(y, C)
```

# Defining Tensorflow Operations
## Comparison Operations
* Useful for comparing two tensors. 
* Let us comsider two example tensors, `x` and `y`.

```
# Let us assume the following values for x and y
# x (2D tensor) => [[1, 2], [3, 4]]
# y (2D tensor) => [[4, 3], [3, 2]]
x = tf.constant([[1, 2], [3, 4]], dtype=tf.int32)
y = tf.constant([[4, 3], [3, 2]], dtype=tf.int32)

# Checks if two tensors are equal element-wise and returns a boolean tensor
# x_equal_y => [[False, False], [True, False]]
x_equal_y = tf.equal(x, y, name=None)

# Checks if x is less than y element-wise and returns a boolean tensor
# x_less_y => [[True, True], [False, False]]
x_less_y = tf.less(x, y, name=None)

# Checks if x is greater than or equal to y element-wise and returns a boolean tensor
# x_great_y => [[False, False], [True, True]]
x_great_y = tf.greater_equal(x, y, name=None)

# Selects elements from x and y depending on whether
# the condition is satisfied (select elements from x)
# or the condition failed (select elements from y)
conditon = tf.constant([[True, False], [True, False]], dtype=tf.bool)
# x_cond_y => [[1, 3], [3, 2]]
x_cond_y = tf.where(condition, x, y, name=None)
``` 

## Mathematical Operations
* Can perform simple to complex math operations on tensors. 

In [1]:
import tensorflow as tf
import numpy as np

In [0]:
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)

In [0]:
x = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
y = tf.constant([[4, 3], [3, 2]], dtype=tf.float32)

In [17]:
# Element-wise addition
x_add_y = session.run(tf.add(x, y))
print(x_add_y)

[[5. 5.]
 [6. 6.]]


In [18]:
# Matrix multiplication (not element-wise)
x_mul_y = session.run(tf.matmul(x, y))
print(x_mul_y)

[[10.  7.]
 [24. 17.]]


In [19]:
# Compute natural logarithm of x element-wise (ln(x))
ln_x = session.run(tf.log(x))
print(ln_x)

[[0.        0.6931472]
 [1.0986123 1.3862944]]


In [20]:
# Peform reduction operation across specified axis
x_sum_1 = session.run(tf.reduce_sum(x, axis=[1], keepdims=False))
print(x_sum_1)

[3. 7.]


In [21]:
x_sum_2 = session.run(tf.reduce_sum(x, axis=[0], keepdims=True))
print(x_sum_2)

[[4. 6.]]


In [22]:
# Segments tensor according to segment_ids (items with same id in the same
# segment) and computes a segmented sum of the data
data = tf.constant([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=tf.float32)
segment_ids = tf.constant([0, 0, 0, 1, 1, 2, 2, 2, 2, 2], dtype=tf.int32)
x_seg_sum = session.run(tf.segment_sum(data, segment_ids))
print(x_seg_sum)


[ 6.  9. 40.]


In [0]:
session.close()

# Scatter and gather operations
* Play a vital role in matrix manipulation tasks
* Only way to index tensors in TensorFlow
* You cannot access elements of tensors in TensorFlow as you would in NumPy (e.g `x[1, 0]` where `x` is a 2D `numpy.ndarray`). 
* Scatter operation - allows you to assign values to specific indices of a given tensor
* Gather operation - allows you to extract a slice (or individual elements) of a given tensor.

In [27]:
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)

/tensorflow-1.15.0/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [38]:
# 1D scatter operation
ref = tf.Variable(tf.constant([1, 9, 3, 10, 5], dtype=tf.float32),
                  name='scatter_update')
indices = [1, 3]
updates = tf.constant([2, 4], dtype=tf.float32)
tf.global_variables_initializer().run()
tf_scatter_update = tf.scatter_update(ref, indices, updates,
                                      use_locking=None, name=None)
tf_sur = session.run(tf_scatter_update)
print(tf_scatter_update)

Tensor("ScatterUpdate_8:0", shape=(5,), dtype=float32_ref)
